In [1]:
import sys
sys.path.append('../')

from manim import *
from common import nuclear
from importlib import reload
import random
import nltk
nltk.download('punkt')

from manim_voiceover import VoiceoverScene
from manim_voiceover.services.coqui import CoquiService

Manim Community v0.17.3

[nltk_data] Downloading package punkt to /home/mic/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
%%manim -v WARNING --disable_caching -ql MainScene

reload(nuclear)

config.media_width = "75%"
config.media_embed = True

PARTICLE_SIZE_MULTIPLIER = 0.6
NUCLEON_SEPARATION = 1.0
PARTICLE_LABEL_FONT_SIZE = 30
HEADING_FONT_SIZE = 200

pronunciation_fixes = {
    "beta": "bayta"
}

class MainScene(VoiceoverScene):
    def construct(self):
        # Set up voiceover
        # set the speech synthesizer for the voiceover
        self.set_speech_service(CoquiService())
        
        self.decays_animations()

    def voiceover_fixed_pronunciation(self, text):
        tokens = nltk.word_tokenize(text)

        fixed_tokens = []
        for token in tokens:
            if token.lower() in pronunciation_fixes:
                fixed_tokens.append(pronunciation_fixes[token])
            else:
                fixed_tokens.append(token)
                
        text = " ".join(fixed_tokens)
        
        return self.voiceover(text)
            
    def decays_animations(self):
        with self.voiceover_fixed_pronunciation("""
          There are 4 main types of decay that AQA includes in the
  specification: alpha, beta minus, beta plus and gamma. Some other
  decays may come up, but they are not mentioned on the specification.
        """) as tracker:
            ## Prepare the scene
            nucleus = nuclear.Nucleus().init_from_nums(12, 12, NUCLEON_SEPARATION, 4, PARTICLE_SIZE_MULTIPLIER, seed=6)
            nucleus.shift(DOWN)

            # show the nucleus on screen
            self.play(*nucleus.create_anims(Create))
            
            self.wait()
            
            ## Alpha decay
            heading = Tex(r"$\alpha$ decay", font_size=HEADING_FONT_SIZE).shift(2.5 * UP)
            self.play(Write(heading))
            daughter1, daughter2, pairs1, pairs2 = nucleus.decay(2, 2, (2, -1), False, False, 1)
    
            # Move the bigger daughter nucleus to where we started to avoid weird shifts
            daughter2.move_to(nucleus)
            daughter1.shift(4 * RIGHT + 2 * DOWN)
            
            self.play(*map(lambda p: ReplacementTransform(p[0], p[1]), pairs1 + pairs2))
    
            self.wait()
    
            # Clean up
            self.play(*daughter1.create_anims(Uncreate))  # hide the alpha particle
            nucleus = daughter2
            
            # Beta - decay
            self.beta_decay(False, nucleus, heading)

            # Beta + decay
            self.beta_decay(True, nucleus, heading)
            
            # Gamma decay
            heading_gamma = Tex(r"$\gamma$ decay", font_size=HEADING_FONT_SIZE).shift(2.5 * UP)
            self.play(Transform(heading, heading_gamma))
            new_nucleons = nucleus.get_nucleons_list()
            random.shuffle(new_nucleons)
            new_nucleus = nuclear.Nucleus().init_from_nucleons(new_nucleons, NUCLEON_SEPARATION, 4, PARTICLE_SIZE_MULTIPLIER, False)
            new_nucleus.move_to(nucleus)
            # Find some point on the edge of the nucleus
            
            gamma_init_coords = nucleus.find_closest_nucleon((0, 0)).get_center()
            initial_arrow = Arrow(start=gamma_init_coords, end=gamma_init_coords)
            final_arrow = Arrow(start=gamma_init_coords, end=(4 * RIGHT + 2 * UP))
            # Move the gamma particle with the arrow
            gamma_label = MathTex(r"\gamma", font_size=PARTICLE_LABEL_FONT_SIZE)
            gamma_label.move_to(initial_arrow)
            gamma_label.add_updater(lambda x: x.move_to(initial_arrow.point_from_proportion(0.7)).shift(0.3 * UP))
            self.add(initial_arrow, gamma_label)
            # Go from each original nucleon to the shuffled one
            self.play(ReplacementTransform(initial_arrow, final_arrow), 
                      *map(lambda n: ReplacementTransform(n[0], n[1]),
                           zip(new_nucleons, new_nucleus.get_nucleons_list())))
            self.wait(1)

    def beta_decay(self, is_plus, nucleus, heading):
        if is_plus:
            sign_letter = "+"
            nucleon_type = nuclear.PROTON
            new_nucleon_type = nuclear.NEUTRON
            beta_type = nuclear.POSITRON
            neutrino_label = r"\nu_e"
        else:
            sign_letter = "-"
            nucleon_type = nuclear.NEUTRON
            new_nucleon_type = nuclear.PROTON
            beta_type = nuclear.ELECTRON
            neutrino_label = r"\bar{\nu_e}"
        
        new_heading = Tex(f"$\\beta^{sign_letter}$ decay", font_size=HEADING_FONT_SIZE).shift(2.5 * UP)
        self.play(Transform(heading, new_heading))
        
        nucleon = nucleus.find_closest_nucleon((-2, 0), nucleon_type)

        beta = nuclear.Particle(beta_type, PARTICLE_SIZE_MULTIPLIER)
        beta.move_to(nucleon)

        beta_label1 = nuclear.ParticleLabel(beta, PARTICLE_LABEL_FONT_SIZE, f"\\beta^{sign_letter}")
        beta_label2 = nuclear.ParticleLabel(beta, PARTICLE_LABEL_FONT_SIZE)

        neutrino = nuclear.Particle(nuclear.NEUTRINO, PARTICLE_SIZE_MULTIPLIER)
        neutrino_label = nuclear.ParticleLabel(neutrino, PARTICLE_LABEL_FONT_SIZE, neutrino_label, color=DARK_GRAY)
        neutrino.move_to(nucleon)

        self.add(beta, beta_label1, neutrino, neutrino_label)
        self.play(nucleon.animate.set_particle_type(new_nucleon_type), beta.animate.shift(2 * LEFT + DOWN),
                  neutrino.animate.shift(2.5 * LEFT + UP), run_time=1)

        self.wait()

        self.play(ReplacementTransform(beta_label1, beta_label2))
        
        self.wait()

        # Clean up
        self.play(Uncreate(beta), Uncreate(beta_label2),
                  Uncreate(neutrino), Uncreate(neutrino_label))  # destroy the beta particle

 > tts_models/en/ljspeech/tacotron2-DDC is already downloaded.
 > vocoder_models/en/ljspeech/hifigan_v2 is already downloaded.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio P